In [ ]:
# installing the libraries for transformers
#!pip install -U -q sentence-transformers transformers bitsandbytes accelerate sentencepiece

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Deep Learning library
import torch

# to load transformer models
#from sentence_transformers import SentenceTransformer
#from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline # T5 is Google

# to split the data
from sklearn.model_selection import train_test_split

# to compute performance metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


In [ ]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))
misconceptions_filename = files[1]
train_filename = files[2]
test_filename = files[3]
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## File and Field Information
### [train/test].csv

* QuestionId - Unique question identifier (int).
* ConstructId - Unique construct identifier (int) .
* ConstructName - Most granular level of knowledge related to question (str).
* CorrectAnswer - A, B, C or D (char).
* SubjectId - Unique subject identifier (int).
* SubjectName - More general context than the construct (str).
* QuestionText - Question text extracted from the question image using human-in-the-loop OCR (str) .
* Answer[A/B/C/D]Text - Answer option A text extracted from the question image using human-in-the-loop OCR (str).
* Misconception[A/B/C/D]Id - Unique misconception identifier (int). Ground truth labels in train.csv; your task is to predict these labels for test.csv.

### misconception_mapping.csv
maps MisconceptionId to its MisconceptionName

### sample_submission.csv
A submission file in the correct format.
* QuestionId_Answer - Each question has three incorrect answers for which need you predict the MisconceptionId.
* MisconceptionId - You can predict up to 25 values, space delimited.

In [ ]:
train_data = pd.read_csv(train_filename)
train_data_orig = train_data.copy()
train_data.head()

In [ ]:
print ("train_data shape")
print("rows",train_data.shape[0])
print("columns",train_data.shape[1])

In [ ]:
subject_groups = train_data.groupby(["SubjectId","SubjectName"]).count().sort_values('QuestionId',ascending=False).reset_index()
print(subject_groups.shape[0], " different subject groups")
subject_groups.head()

In [ ]:
print("percentages of null values")
pd.DataFrame({'Count':train_data.isnull().sum()[train_data.isnull().sum()>0],'Percentage':(train_data.isnull().sum()[train_data.isnull().sum()>0]/train_data.shape[0])*100})

In [ ]:
# records where more than one misconception id is missing
missing_a_misconpception = train_data[(train_data.CorrectAnswer == "A") & (train_data.MisconceptionCId.isnull()) ]
missing_a_misconpception.head()

In [ ]:
misc_con_data = pd.read_csv(misconceptions_filename)
misc_con_data.head()

In [ ]:
# defining a function to compute the cosine similarity between two embedding vectors
def cosine_score(text):
    # encoding the text
    embeddings = model.encode(text)

    # calculating the L2 norm of the embedding vector
    norm1 = np.linalg.norm(embeddings[0])
    norm2 = np.linalg.norm(embeddings[1])

    # computing the cosine similarity
    cosine_similarity_score = ((np.dot(embeddings[0],embeddings[1]))/(norm1*norm2))

    return cosine_similarity_score

In [ ]:
# use pylatexenc to convert latex to text then vector